In [1]:
# Full runnable code for the StoryFlowAgent example
from typing import AsyncGenerator
from typing_extensions import override

from loguru import logger
from pydantic import BaseModel, Field
from google.adk.agents import LlmAgent, BaseAgent, LoopAgent, SequentialAgent
from google.adk.agents.invocation_context import InvocationContext
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.events import Event

In [2]:
from typing import AsyncGenerator
from typing_extensions import override

from google.adk.agents import BaseAgent
from google.adk.events.event import Event
from google.genai import types
from google.adk.agents.invocation_context import InvocationContext


In [4]:
# --- Constants ---
APP_NAME = "story_app"
USER_ID = "12345"
SESSION_ID = "123344"
GEMINI_2_FLASH = "gemini-2.0-flash"

In [5]:
agent = DummyAgent(name="dummy")

In [6]:
# --- Setup Runner and Session ---
session_service = InMemorySessionService()
initial_state = {"topic": "a brave kitten exploring a haunted house"}
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state=initial_state # Pass initial state here
)
logger.info(f"Initial session state: {session.state}")

runner = Runner(
    agent=agent, # Pass the custom orchestrator agent
    app_name=APP_NAME,
    session_service=session_service
)


2025-06-25 03:10:04.818 | INFO     | __main__:<module>:10 - Initial session state: {'topic': 'a brave kitten exploring a haunted house'}


In [7]:
async def call_agent(user_input_topic: str):
    """
    Sends a new topic to the agent (overwriting the initial one if needed)
    and runs the workflow.
    """
    content = types.Content(role='user', parts=[types.Part(text=f"Generate a story about: {user_input_topic}")])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    final_response = "No final response captured."
    for event in events:
        if event.is_final_response() and event.content and event.content.parts:
            logger.info(f"Potential final response from [{event.author}]: {event.content.parts[0].text}")
            final_response = event.content.parts[0].text

    print("\n--- Agent Interaction Result ---")
    print("Agent Final Response: ", final_response)

    final_session = await session_service.get_session(app_name=APP_NAME, 
                                                user_id=USER_ID, 
                                                session_id=SESSION_ID)
    print("Final Session State:")
    import json
    print(json.dumps(final_session.state, indent=2))
    print("-------------------------------\n")

In [8]:
await call_agent("a lonely robot finding a friend in a junkyard")


2025-06-25 03:10:04.828 | INFO     | __main__:call_agent:12 - Potential final response from [dummy]: Ok



--- Agent Interaction Result ---
Agent Final Response:  Ok
Final Session State:
{
  "topic": "a brave kitten exploring a haunted house"
}
-------------------------------

